<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

**Imports and Setup. The initialization cell below should run automatically and print out "Done with initialization!"**

In [ ]:
from esper.prelude import *
import numpy as np
from IPython.display import display, clear_output
import ipywidgets as widgets
import datetime
from rekall.interval_list import IntervalList
from rekall.temporal_predicates import overlaps, overlaps_before
from query.models import Labeler, Shot

try:
    VIDEO
except NameError:
    VIDEO = None
    STARTING_FRAME = None
    ENDING_FRAME = None
    ESPER_WIDGET = None
    SELECTED_HARD_CUTS = []
    SELECTED_TRANSITION_FRAMES = []

def choose_random_segment():
    video = Video.objects.filter(ignore_film=False).order_by('?')[0]
    five_minutes = int(60 * 5 * video.fps)
    one_minute = int(60 * video.fps)
    starting_frame = np.random.randint(
        five_minutes, video.num_frames - 2 * five_minutes)

    global VIDEO, STARTING_FRAME, ENDING_FRAME
    VIDEO = video
    STARTING_FRAME = starting_frame
    ENDING_FRAME = STARTING_FRAME + one_minute - 1

    print('Selected film {} ({}), frames {}-{}'.format(
        video.title, video.year, STARTING_FRAME, ENDING_FRAME))

# replaces qs_to_result


def frame_result(video_id, frame_nums):
    materialized_result = []
    for frame_num in frame_nums:
        materialized_result.append({
            'video': video_id,
            'min_frame': frame_num,
            'objects': []
        })
    return {'type': 'frames', 'count': 0, 'result': [{
        'type': 'flat', 'label': '', 'elements': [mr]
    } for mr in materialized_result]}


def cache_labels(b):
    global SELECTED_HARD_CUTS, SELECTED_TRANSITION_FRAMES
    SELECTED_HARD_CUTS = ESPER_WIDGET.selected
    SELECTED_TRANSITION_FRAMES = ESPER_WIDGET.ignored
    print("Saved {}".format(datetime.datetime.now()))


def label_segment():
    global VIDEO, ESPER_WIDGET, STARTING_FRAME, ENDING_FRAME
    frames = frame_result(VIDEO.id, range(STARTING_FRAME, ENDING_FRAME + 1))

    save = widgets.Button(
        description='Save Progress',
        disabled=False,
        button_style='success',
        tooltip='Save Progress'
    )
    ESPER_WIDGET = esper_widget(
        frames,
        show_paging_buttons=True,
        jupyter_keybindings=True,
        results_per_page=48,
        thumbnail_size=0.75,
        selected_cached=SELECTED_HARD_CUTS,
        ignored_cached=SELECTED_TRANSITION_FRAMES,
        max_width=965
    )
    display(ESPER_WIDGET)
    display(save)
    save.on_click(cache_labels)


def inspect_hard_cuts():
    global SELECTED_HARD_CUTS, STARTING_FRAME
    frame_nums = [f + STARTING_FRAME for f in SELECTED_HARD_CUTS]

    frames = frame_result(VIDEO.id, frame_nums)

    update = widgets.Button(
        description='Update',
        disabled=False,
        button_style='success',
        tooltip='Update'
    )
    esp = esper_widget(
        frames,
        show_paging_buttons=True,
        jupyter_keybindings=True,
        results_per_page=48,
        thumbnail_size=0.75,
        max_width=965,
        selected_cached=[],
        ignored_cached=[]
    )
    display(esp)
    display(update)

    def update_hard_cuts(b):
        global SELECTED_HARD_CUTS
        deselected_frames = [
            frame_nums[i]
            for i in esp.ignored
        ]

        hard_cuts_to_remove = [
            selection
            for frame_num, selection in zip(frame_nums, SELECTED_HARD_CUTS)
            if frame_num in deselected_frames
        ]

        SELECTED_HARD_CUTS = [
            cut for cut in SELECTED_HARD_CUTS
            if cut not in hard_cuts_to_remove
        ]

        clear_output()
        inspect_hard_cuts()

    update.on_click(update_hard_cuts)


def inspect_transitions():
    global SELECTED_TRANSITION_FRAMES, STARTING_FRAME
    frame_nums = [f + STARTING_FRAME for f in SELECTED_TRANSITION_FRAMES]

    frames = frame_result(VIDEO.id, frame_nums)

    update = widgets.Button(
        description='Update',
        disabled=False,
        button_style='success',
        tooltip='Update'
    )
    esp = esper_widget(
        frames,
        show_paging_buttons=True,
        jupyter_keybindings=True,
        results_per_page=48,
        thumbnail_size=0.75,
        max_width=965,
        selected_cached=[],
        ignored_cached=[]
    )
    display(esp)
    display(update)

    def update_transitions(b):
        global SELECTED_TRANSITION_FRAMES
        deselected_frames = [
            frame_nums[i]
            for i in esp.ignored
        ]

        transition_frames_to_remove = [
            selection
            for frame_num, selection in zip(frame_nums, SELECTED_TRANSITION_FRAMES)
            if frame_num in deselected_frames
        ]

        SELECTED_TRANSITION_FRAMES = [
            cut for cut in SELECTED_TRANSITION_FRAMES
            if cut not in transition_frames_to_remove
        ]

        clear_output()
        inspect_transitions()

    update.on_click(update_transitions)


def prepare_orm_objects():
    global STARTING_FRAME, SELECTED_HARD_CUTS, SELECTED_TRANSITION_FRAMES, VIDEO
    selected_shot_boundaries = [
        STARTING_FRAME + idx for idx in SELECTED_HARD_CUTS]
    transition_frames = IntervalList([
        (STARTING_FRAME + f, STARTING_FRAME + F, 0)
        for f in SELECTED_TRANSITION_FRAMES
    ]).dilate(1).coalesce().dilate(-1)
    selected_shot_boundaries = sorted(selected_shot_boundaries + [
        int((transition.end + transition.start) / 2)
        for transition in transition_frames.get_intervals()
    ])

    shots = []
    for i in range(0, len(selected_shot_boundaries) - 1):
        shots.append(
            (selected_shot_boundaries[i], selected_shot_boundaries[i+1] - 1, {}))
    shots_intrvllist = IntervalList(shots)

    shots_with_transition_info = shots_intrvllist.join(
        transition_frames,
        predicate=overlaps(),
        merge_op=lambda shot, transition: (
            shot.start, shot.end,
            {'transition_start_max_frame': transition.end}
            if overlaps_before()(transition, shot) else
            {'transition_end_min_frame': transition.start}
        )
    ).set_union(
        shots_intrvllist
    ).coalesce(payload_merge_op=lambda p1, p2: {**p1, **p2})

    labeler_name = 'shot-manual-{}'.format(
        datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
    )
    new_labeler, _ = Labeler.objects.get_or_create(name=labeler_name)
    print('Labeler created:', labeler_name)

    new_shots = []
    for intrvl in shots_with_transition_info.get_intervals():
        new_shot = Shot(
            min_frame=intrvl.start,
            max_frame=intrvl.end,
            video=VIDEO,
            labeler=new_labeler
        )
        if 'transition_start_max_frame' in intrvl.payload:
            new_shot.transition_in_max_frame = intrvl.payload['transition_start_max_frame']
        if 'transition_end_min_frame' in intrvl.payload:
            new_shot.transition_out_min_frame = intrvl.payload['transition_end_min_frame']
        new_shots.append(new_shot)

    return new_labeler, new_shots


def save_orm_objects(shots):
    print('Saving shots...')
    with transaction.atomic():
        Shot.objects.bulk_create(shots)
    print('Done!')


def reset_notebook_state():
    global VIDEO, STARTING_FRAME, ENDING_FRAME, \
        ESPER_WIDGET, SELECTED_HARD_CUTS, SELECTED_TRANSITION_FRAMES
    VIDEO = None
    STARTING_FRAME = None
    ENDING_FRAME = None
    ESPER_WIDGET = None
    SELECTED_HARD_CUTS = []
    SELECTED_TRANSITION_FRAMES = []

print('Done with initialization!')

Step 1: Run the cell below to pick a random five-minute segment from a film to label.

In [ ]:
choose_random_segment()

Step 2: Run the cell below to show the labeling interface.

For **hard cuts**, hover over the frame and use `[` to mark the first frame of the new shot.

For **fades/wipes/etc**, use `]` to mark all the frames in the transition (hover over the frames and press `]`).

You can hover over a frame and use `=` to expand the frame to inspect it closer, or `Shift-P` to play the film starting from that frame.

You can click the Save button at any time to locally cache your labels - these will persist across refreshes, but not across kernel deaths.

When you're ready to move on, click the Save button.

In [ ]:
label_segment()

Step 3: Run the cell below to inspect your hard cut labels.

Use `]` to **deselect** any frames that you'd like to remove from your labelling pass. If you'd like to add more frames, go back to the previous cell. Your previous work should be cached.

In [ ]:
inspect_hard_cuts()

Step 4: Run the cell below to inspect your transition labels.

Use `]` to **deselect** any frames that you'd like to remove from your labelling pass. If you'd like to add more frames, go back to Step 2. Your previous work should be cached

In [ ]:
inspect_transitions()

Step 5: Run the two cells below to commit your labels to the database!

In [ ]:
labeler, shots = prepare_orm_objects()

In [ ]:
save_orm_objects(shots)

Step 6: Run the cell below to reset the notebook state and start over with a new segment from Step 1!

In [ ]:
reset_notebook_state()